# JC Penny Store Closings


## Workflow

Investigate JC Penny store closings$^1$  by:

* Tagging locations as Urban vs Rural (using population density from the Data Observatory)
* Draw 10 minutes walk or drive isochrones based on whether the location is urban or not
* Visualize data with cartoframes
* Augment isochrones with Data Observatory measures
* Visualize data in Builder and add widgets for specific measures and store properties

Final dashboard: https://team.carto.com/u/eschbacher/builder/0592fcae-3026-11e7-b861-0e3ebc282e83/embed

1. closing status is real, but the actual close date is chosen randomly from the last five years

## Installing dependencies

Install [cartoframes](https://github.com/cartodb/cartoframes) (which is currently in beta). I recommend installing in a virtual environment to keep things clean and sandboxed.

## Getting the data

Download the JC Penny store location data from here:
* <http://eschbacher.carto.com/api/v2/sql?q=select%20*%20from%20jc_penny_stores&format=csv&filename=jc_penny_stores>

## Workflow for obtaining data

Pull JC Penny locations from my CARTO account into cartoframes

In [2]:
import pandas as pd
import cartoframes
import json
import warnings
warnings.filterwarnings('ignore')

USERNAME = 'skickham' # <-- Put your carto username here
APIKEY = '79c4ab7de1e6c6794a3b84b00ff0bf45659ce668' # <-- Put your carto api key here

# use cartoframes.credentials.set_creds() to save credentials for future use
cc = cartoframes.CartoContext(api_key=APIKEY,
                              base_url='https://{}.carto.com/'.format(USERNAME))
table_name = 'jc_penny_stores'

# load JC Penny locations into DataFrame
df = cc.read(table_name)
df.head()

,status,distance,allhomes_zhvi_predenom_2016_08,the_geom_webmercator,name,state_abb,total_pop_area_2011_2015,closed_date,phone,state,address,the_geom
cartodb_id,,,,,,,,,,,,
405,retaining,24140.0,NaN,0101000020110F0000D4179582CCE761C14E29ED592A54...,JCPenney - Richmond Centre,KY,368.266243,NaT,(859) 626-7857,Kentucky,2037 Lantern Ridge Dr,0101000020E6100000E1EF17B3251555C04CE2AC889ADE...
3,closing,24140.0,NaN,0101000020110F00006F876C7D722462C10CFF22F21C57...,JCPenney - Colonial University in Auburn,AL,866.007594,2014-04-17,(334) 826-1954,Alabama,1627 Opelika Rd Ste 69,0101000020E61000000AD6389B8E5C55C07619FED30D50...
4,closing,24140.0,155500.0,0101000020110F00003AD009EF347862C110A1BA13DE0E...,JCPenney - Clnl Prom @tannehill in Bessemer,AL,174.355475,2015-07-21,(205) 565-2720,Alabama,4835 Promenade Pkwy,0101000020E6100000E96514CB2DBF55C07F2F8507CDAA...
5,retaining,24140.0,NaN,0101000020110F0000E4EBC3C39B6E62C139F61E854967...,JCPenney - Cullman Shopg Cntr,AL,6.903157,NaT,(256) 739-3726,Alabama,711 Second Ave NW,0101000020E61000003468E89FE0B355C01475E61E1258...
6,retaining,24140.0,NaN,0101000020110F0000D2E0C270202362C171737DD2BBF7...,JCPenney - Wiregrass Commons in Dothan,AL,494.424072,NaT,(334) 794-0531,Alabama,900 Commons Dr Ste 900,0101000020E6100000BEF6CC92005B55C0AF44A0FA0741...


## JC Penny Store Closings

* Purple = stores closing
* Orange = stores staying open

In [3]:
from cartoframes import Layer   # can map my layer inside python notebook 
from cartoframes.styling import vivid

cc.map(layers=Layer(table_name,
                    color={'column': 'status', 'scheme': vivid(10, 'category')}),
       interactive=False)

## Augment with DO to get 'urban-ness' metric (population density)

In [6]:
# get population, other measures within 5 minute walk time
# More info about this Data Observatory measure here:
#  https://cartodb.github.io/bigmetadata/united_states/age_gender.html#total-population
df = cc.data_augment(table_name, [{'numer_id': 'us.census.acs.B01003001',
                                   'normalization': 'area',
                                   'numer_timespan': '2011 - 2015'}])
df.head()

,status,distance,allhomes_zhvi_predenom_2016_08,the_geom_webmercator,name,state_abb,total_pop_area_2011_2015,closed_date,phone,state,address,the_geom
cartodb_id,,,,,,,,,,,,
394,retaining,24140.0,NaN,0101000020110F0000720A5E60110862C1270A265C5B4E...,JCPenney - Bradford Sq in Hopkinsville,KY,28.082250,NaT,(270) 886-9441,Kentucky,4000 Ft Campbell Blvd,0101000020E61000006AD95A5F243B55C08104C58F31D9...
570,retaining,24140.0,NaN,0101000020110F0000354B5FE848B163C192BE1767EEA8...,JCPenney - Washington,MO,14.059081,NaT,(636) 390-2405,Missouri,5886 Highway 100,0101000020E61000001FD95C35CF2F57C039F06AB9332D...
591,retaining,24140.0,NaN,0101000020110F000040C4A6BC489368C188BC286C4709...,JCPenney - Elko Shopping Plaza,NV,131.475164,NaT,(775) 738-7274,Nevada,2190 Idaho St,0101000020E610000069C70DBF9BEF5CC00342EBE1CB6C...
2,retaining,24140.0,NaN,0101000020110F0000C49A63207B5D62C118FDB9263807...,JCPenney - Covington Mall in Andalusia,AL,152.558767,NaT,(334) 222-8484,Alabama,922 River Falls St,0101000020E61000006D7022FAB59F55C0A4213E0B9D50...
1,retaining,24140.0,153300.0,0101000020110F000096B58E53286E62C12515EDDFD8F1...,JCPenney - Colonial Promenade in Alabaster,AL,62.527422,NaT,(205) 663-2763,Alabama,340 S Colonial Dr,0101000020E6100000323CF6B358B355C02D5BEB8B849C...


## Get a sense of the range of data

In [7]:
df.describe()

,distance,allhomes_zhvi_predenom_2016_08,total_pop_area_2011_2015
count,1078.0,7.070000e+02,1078.000000
mean,24140.0,2.235372e+05,696.898505
std,0.0,1.588468e+05,1724.807367
min,24140.0,4.380000e+04,0.000000
25%,24140.0,1.302000e+05,57.923748
50%,24140.0,1.830000e+05,347.338072
75%,24140.0,2.480500e+05,814.271287
max,24140.0,1.765800e+06,22221.862461


## Create isochrones based on travel inferences

Create a derivative table with geometries as isochrones of walk/drive times from store locations. If pop density is above 5000 people / sq. km., assume it's a walkable area. Otherwise, assume cars are the primary mode of transit.

**Note:** This functionality is a planned cartoframes method.

In [21]:
%%time
df = cc.query('''
    SELECT 
        CASE WHEN total_pop_area_2011_2015 > 5000
             THEN ST_Buffer(the_geom::geography, 800)::geometry
             ELSE ST_Buffer(the_geom::geography, 1200)::geometry
             END as the_geom,
        {keep_columns}
    FROM
        {table_name}
             '''.format(table_name=table_name,
                        keep_columns=', '.join(set(df.columns) - {'the_geom', 'the_geom_webmercator'})),
             table_name=(table_name + '_isochrones'))

CartoException: [u'column "median_income_2011_2015" does not exist']

If this fails because of a lack of credits (i.e., reaching quota), then replace the `(cdb_isochrone(the_geom, 'walk', Array[600])).the_geom` pieces with `ST_Buffer(the_geom::geography, 800)::geometry` for an approximate 10 minute walk ('crow flies' distance), and `ST_Buffer(the_geom::geography, 12000)::geometry` for an approximate 10 minute drive (assuming 45 mph on average for 10 minutes).

In [14]:
iso_table_name = (table_name + '_isochrones')

In [ ]:
ccwrite()

In [11]:
df.head()

,status,distance,allhomes_zhvi_predenom_2016_08,the_geom_webmercator,name,state_abb,total_pop_area_2011_2015,closed_date,phone,state,address,the_geom
cartodb_id,,,,,,,,,,,,
1,retaining,24140.0,NaN,0103000020110F000001000000210000007EC70FA3AC00...,JCPenney - Bradford Sq in Hopkinsville,KY,28.082250,NaT,(270) 886-9441,Kentucky,4000 Ft Campbell Blvd,0103000020E61000000100000021000000C1E021C96F32...
2,retaining,24140.0,NaN,0103000020110F00000100000021000000DF55ECDED1A9...,JCPenney - Washington,MO,14.059081,NaT,(636) 390-2405,Missouri,5886 Highway 100,0103000020E6100000010000002100000031E2D7130527...
3,retaining,24140.0,NaN,0103000020110F00000100000021000000B4F9B1548C8B...,JCPenney - Elko Shopping Plaza,NV,131.475164,NaT,(775) 738-7274,Nevada,2190 Idaho St,0103000020E61000000100000021000000E40DF6EF7FE6...
4,retaining,24140.0,NaN,0103000020110F0000010000002100000091577154A056...,JCPenney - Covington Mall in Andalusia,AL,152.558767,NaT,(334) 222-8484,Alabama,922 River Falls St,0103000020E61000000100000021000000EF6044CFA397...
5,retaining,24140.0,153300.0,0103000020110F000001000000210000006288F4582867...,JCPenney - Colonial Promenade in Alabaster,AL,62.527422,NaT,(205) 663-2763,Alabama,340 S Colonial Dr,0103000020E61000000100000021000000103573C11AAB...


In [16]:
from cartoframes import BaseMap
cc.map(layers=[BaseMap('light'),
               Layer(table_name + '_isochrones'),
               Layer(table_name)],
       zoom=12, lng=-73.9668, lat=40.7306,
       interactive=False) # or True

In [22]:
# show choropleth of isochrones by pop density
from cartoframes.styling import vivid
cc.map(layers=[Layer(table_name + '_isochrones',
                    color='total_pop_area_2011_2015'),
              Layer(table_name, size=6, color={'column': 'status', 'scheme': vivid(2)})],
       zoom=8, lng=-74.7729, lat=39.9771,
       interactive=False)  #or can do True

In [18]:
# Data Observatory measures: median income, male age 30-34 (both ACS)
# Male age 30-34: https://cartodb.github.io/bigmetadata/united_states/age_gender.html#male-age-30-to-34
# Median Income: https://cartodb.github.io/bigmetadata/united_states/income.html#median-household-income-in-the-past-12-months

# Note: this may take a minute or two because all the measures are being calculated based on the custom geographies
#       that are passed in using spatially interpolated calculations (area-weighted measures)

data_obs_measures = [{'numer_id': 'us.census.acs.B01001012'},
                     {'numer_id': 'us.census.acs.B19013001'}]
df = cc.data_augment(table_name + '_isochrones', data_obs_measures)
df.head()

,status,distance,allhomes_zhvi_predenom_2016_08,the_geom_webmercator,name,state_abb,male_30_to_34_2011_2015,total_pop_area_2011_2015,closed_date,phone,state,address,median_income_2011_2015,the_geom
cartodb_id,,,,,,,,,,,,,,
1,retaining,24140.0,NaN,0103000020110F000001000000210000007EC70FA3AC00...,JCPenney - Bradford Sq in Hopkinsville,KY,0.032911,28.082250,NaT,(270) 886-9441,Kentucky,4000 Ft Campbell Blvd,41263.301112,0103000020E61000000100000021000000C1E021C96F32...
2,retaining,24140.0,NaN,0103000020110F00000100000021000000DF55ECDED1A9...,JCPenney - Washington,MO,0.034218,14.059081,NaT,(636) 390-2405,Missouri,5886 Highway 100,41750.934264,0103000020E6100000010000002100000031E2D7130527...
3,retaining,24140.0,NaN,0103000020110F00000100000021000000B4F9B1548C8B...,JCPenney - Elko Shopping Plaza,NV,0.043822,131.475164,NaT,(775) 738-7274,Nevada,2190 Idaho St,74506.782991,0103000020E61000000100000021000000E40DF6EF7FE6...
4,retaining,24140.0,NaN,0103000020110F0000010000002100000091577154A056...,JCPenney - Covington Mall in Andalusia,AL,0.027246,152.558767,NaT,(334) 222-8484,Alabama,922 River Falls St,36108.890613,0103000020E61000000100000021000000EF6044CFA397...
5,retaining,24140.0,153300.0,0103000020110F000001000000210000006288F4582867...,JCPenney - Colonial Promenade in Alabaster,AL,0.036400,62.527422,NaT,(205) 663-2763,Alabama,340 S Colonial Dr,69592.437593,0103000020E61000000100000021000000103573C11AAB...


## Visualize isochrones based on Data Observatory measure

In [23]:
cc.map(layers=Layer(table_name + '_isochrones',
                    color='median_income_2011_2015'),
       zoom=8, lng=-74.3115, lat=40.1621,
       interactive=False)

## Builder Dashboard

https://team.carto.com/u/eschbacher/builder/0592fcae-3026-11e7-b861-0e3ebc282e83/embed

In [24]:
from IPython.display import HTML
HTML('<iframe width="100%" height="520" frameborder="0" src="https://team.carto.com/u/eschbacher/builder/0592fcae-3026-11e7-b861-0e3ebc282e83/embed" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>')